<a href="https://colab.research.google.com/github/AbhishekAjoy/VaderBot/blob/master/chatbot2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# bot.py

In [ ]:
import requests
import json
import configparser as cfg

class telegram_chatbot():
	def __init__(self, config):
		self.token = self.read_token_from_config_file(config)
		self.base = "https://api.telegram.org/bot{}/".format(self.token)

	def get_updates(self, offset=None):
		url = self.base + "getUpdates?timeout=100"
		if offset:
		    url = url + "&offset={}".format(offset + 1)
		r = requests.get(url)
		return json.loads(r.content)

	def send_message(self, msg, chat_id):
		url = self.base + "sendMessage?chat_id={}&text={}".format(chat_id, msg)
		if msg is not None:
		    requests.get(url)

	def read_token_from_config_file(self, config):
		parser = cfg.ConfigParser()
		parser.read(config)
		return parser.get('creds', 'token')


# chat.py

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [ ]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

44 documents
12 classes ['advice', 'age', 'do', 'goodbye', 'greeting', 'hate', 'location', 'married', 'movie', 'name', 'song', 'who']
64 unique lemmatized words ["'s", 'a', 'advice', 'age', 'am', 'any', 'anyone', 'are', 'be', 'bye', 'call', 'care', 'cya', 'day', 'dislike', 'do', 'favourite', 'from', 'give', 'go', 'good', 'goodbye', 'got', 'hai', 'hate', 'have', 'hello', 'hey', 'hi', 'house', 'how', 'howdy', 'i', 'is', 'jedi', 'later', 'leaving', 'like', 'live', 'married', 'might', 'movie', 'name', 'old', 'padme', 'see', 'should', 'single', 'song', 'starwars', 'ta', 'take', 'there', 'to', 'up', 'vader', 'what', 'whats', 'where', 'which', 'who', 'would', 'you', 'your']


In [ ]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [ ]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
9/9 [==============================] - 0s 1ms/step - loss: 2.4995 - accuracy: 0.0909
Epoch 2/200
9/9 [==============================] - 0s 999us/step - loss: 2.4390 - accuracy: 0.2045
Epoch 3/200
9/9 [==============================] - 0s 1ms/step - loss: 2.3548 - accuracy: 0.2727
Epoch 4/200
9/9 [==============================] - 0s 1ms/step - loss: 2.3045 - accuracy: 0.2500
Epoch 5/200
9/9 [==============================] - 0s 985us/step - loss: 2.2009 - accuracy: 0.3409
Epoch 6/200
9/9 [==============================] - 0s 1ms/step - loss: 2.1094 - accuracy: 0.3409
Epoch 7/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0458 - accuracy: 0.2727
Epoch 8/200
9/9 [==============================] - 0s 1ms/step - loss: 2.0395 - accuracy: 0.3182
Epoch 9/200
9/9 [==============================] - 0s 1ms/step - loss: 1.9610 - accuracy: 0.3409
Epoch 10/200
9/9 [==============================] - 0s 1ms/step - loss: 1.8776 - accuracy: 0.4545
Epoch 11/200
9/9 [=======

In [ ]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [ ]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

#server.py

In [ ]:
#THIS KEEPS RUNNING
bot = telegram_chatbot("/content/config.cfg")


def make_reply(msg):
    reply = None
    if msg is not None:
        reply = chatbot_response(msg)
    return reply

update_id = None
while True:
    updates = bot.get_updates(offset=update_id)
    updates = updates["result"]
    if updates:
        for item in updates:
            update_id = item["update_id"]
            try:
                message = str(item["message"]["text"])
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            reply = make_reply(message)
            bot.send_message(reply, from_)

KeyboardInterrupt: ignored